In [8]:
import tables
import pandas as pd
import glob
import os
import numpy as np
import pickle as pkl

In [2]:
all_pkl_files=[]
pkl_file_sizes=[]
path='/scratch/pdhar/compounding/datasets/google/'
for filename in glob.glob(path+'*pkl'):
    pkl_file_sizes.append(os.path.getsize(filename))
    all_pkl_files.append(filename)
#df = pd.concat(dfs)

In [3]:
pkl_df=pd.DataFrame(all_pkl_files,pkl_file_sizes)
pkl_df.reset_index(inplace=True)
pkl_df.columns=['fsize','fname']

In [4]:
pkl_df['fcat']=pd.cut(pkl_df['fsize'], bins=20,labels=list(range(0,20)))
pkl_df

,fsize,fname,fcat
0,1495,/scratch/pdhar/compounding/datasets/google/uq.pkl,0
1,18896,/scratch/pdhar/compounding/datasets/google/xp.pkl,0
2,8166,/scratch/pdhar/compounding/datasets/google/qc.pkl,0
3,11351153,/scratch/pdhar/compounding/datasets/google/ka.pkl,0
4,636423374,/scratch/pdhar/compounding/datasets/google/br.pkl,0
...,...,...,...
675,99606,/scratch/pdhar/compounding/datasets/google/rb.pkl,0
676,299050,/scratch/pdhar/compounding/datasets/google/vp.pkl,0
677,12756,/scratch/pdhar/compounding/datasets/google/lr.pkl,0
678,619995097,/scratch/pdhar/compounding/datasets/google/ea.pkl,0


In [5]:
pkl_df.groupby('fcat')['fsize'].sum()

fcat
0     30633620567
1     35163156019
2     23338766759
3     26623180403
4      9469649618
5     29925902239
6     20148799914
7     11193956849
8     13286376545
9               0
10    16094653589
11              0
12     9606065976
13              0
14    11563550941
15              0
16              0
17              0
18              0
19    15506249053
Name: fsize, dtype: int64

In [6]:
def write_to_hdf(data_bin):
    df_list=[]
    for row in data_bin.itertuples():
        print(row.fname)
        #tmp_pkl=pd.read_pickle(row.fname)
        #tmp_pkl.reset_index(inplace=True)
        #df_list.append(tmp_pkl)
    #tmp_df=pd.concat(df_list,sort=False)
    #tmp_df.to_hdf(path+'/entire_df_'+str(row.fcat)+'.h5',format='table', key='df',complib='zlib', complevel=5)

In [7]:
pkl_df.groupby('fcat').apply(write_to_hdf)

/scratch/pdhar/compounding/datasets/google/uq.pkl
/scratch/pdhar/compounding/datasets/google/xp.pkl
/scratch/pdhar/compounding/datasets/google/qc.pkl
/scratch/pdhar/compounding/datasets/google/ka.pkl
/scratch/pdhar/compounding/datasets/google/br.pkl
/scratch/pdhar/compounding/datasets/google/os.pkl
/scratch/pdhar/compounding/datasets/google/vo.pkl
/scratch/pdhar/compounding/datasets/google/lm.pkl
/scratch/pdhar/compounding/datasets/google/rz.pkl
/scratch/pdhar/compounding/datasets/google/vh.pkl
/scratch/pdhar/compounding/datasets/google/lj.pkl
/scratch/pdhar/compounding/datasets/google/ey.pkl
/scratch/pdhar/compounding/datasets/google/w_.pkl
/scratch/pdhar/compounding/datasets/google/hx.pkl
/scratch/pdhar/compounding/datasets/google/ak.pkl
/scratch/pdhar/compounding/datasets/google/uv.pkl
/scratch/pdhar/compounding/datasets/google/xw.pkl
/scratch/pdhar/compounding/datasets/google/qd.pkl
/scratch/pdhar/compounding/datasets/google/kf.pkl
/scratch/pdhar/compounding/datasets/google/ot.pkl


""
